# LLM


In [ ]:
!# Install modules
%pip install gradio langchain llama-index openai pypdf

In [ ]:
# Imports
import os
import openai
from gradio import Interface
from gradio.components import Textbox
from llama_index import (
    GPTVectorStoreIndex,
    load_index_from_storage,
    PromptHelper,
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llm_predictor import LLMPredictor
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings

In [ ]:
# Set up environment variables
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-tsp.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "352fe1e9b0be477299c3d6bce68ebf4d"
os.environ["OPENAI_API_VERSION"] = "2023-09-01-preview"

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_version = os.getenv("OPENAI_API_VERSION")

In [ ]:
# Hyperparameters
TEMPERATURE = 0.9
MAX_OUTPUT_LEN = 1024
CONTEXT_WINDOW = 2048
CHUNK_OVERLAP_RATIO = 0.1
LLM_NAME = "gpt-35-turbo"  # Fixed to deployed model
EMBEDDING_MODEL = "text-embedding-ada-002"  # Fixed to deployed model

# Create model and embeddings
llm = LLMPredictor(
    llm=AzureChatOpenAI(
        model=LLM_NAME,
        temperature=TEMPERATURE,
        max_tokens=MAX_OUTPUT_LEN,
    )
)
embed_model = LangchainEmbedding(AzureOpenAIEmbeddings(model=EMBEDDING_MODEL))
prompt_helper = PromptHelper(CONTEXT_WINDOW, MAX_OUTPUT_LEN, CHUNK_OVERLAP_RATIO)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm, embed_model=embed_model, prompt_helper=prompt_helper
)

In [ ]:
def index_dir(dir_path):
    docs = SimpleDirectoryReader(dir_path).load_data()
    index = GPTVectorStoreIndex.from_documents(
        docs, service_context=service_context, prompt_helper=prompt_helper
    )

    index.set_index_id("vector_index")
    index.storage_context.persist(persist_dir="./gpt_store")

    return index


def process_query(input_text):
    storage_context = StorageContext.from_defaults(persist_dir="./gpt_store")
    index = load_index_from_storage(storage_context, service_context=service_context)
    Response = index.as_query_engine().query(input_text)

    return Response.Response


# Index files
index = index_dir("./pdfs/")

# Gradio interface
interface = Interface(
    fn=process_query,
    inputs=Textbox(lines=10, label="Type your question here"),
    outputs=Textbox(lines=10, label="Answer"),
    allow_flagging=False,
    title="Medical Question Expert System",
)
interface.launch(debug=True)

# Analysis of responses


## Prompt 1: How may type 1 diabetes be managed?

### Response (temperature = 0.9):

Based on recent studies, physical activity may play a significant role in managing type 1 diabetes. It has been shown to reduce the risk of severe diabetic retinopathy and premature mortality in patients with type 1 diabetes. Additionally, management may include medication, lifestyle changes such as regular physical activity, adherence to special diets, and regular monitoring. It is important to consult with a healthcare provider to determine an appropriate management plan for type 1 diabetes.

### Response (temperature = 0.6):

Physical activity has been shown to reduce the risk of premature mortality in patients with type 1 diabetes, with or without kidney disease. Additionally, a recent study found that physical activity is associated with a reduced risk of severe diabetic retinopathy in type 1 diabetes. Managing fear of hypoglycemia, adhering to special diets and nutrient recommendations, managing stress and depressive symptoms, and avoiding excessive consumption of alcoholic spirits may also help improve metabolic control and reduce the risk of microvascular complications associated with type 1 diabetes. It is important to consult with a healthcare professional to determine an individualized management plan.

### Response (temperature = 0.3):

Rewrite: One study found that physical activity is associated with a reduced risk of severe diabetic retinopathy in individuals with type 1 diabetes (Tikkanen-Dolenc et al., 2020). Another study found that physical activity reduces the risk of premature mortality in individuals with type 1 diabetes, with or without kidney disease (Groop et al., 2017). These findings suggest that incorporating physical activity into a management plan for type 1 diabetes may be beneficial. However, type 1 diabetes management typically involves a combination of insulin therapy, blood glucose monitoring, healthy eating, physical activity, and regular medical check-ups. It is recommended that individuals with type 1 diabetes work closely with their healthcare team to develop an individualized management plan.

## Prompt 2: Give a detailed summary of the cancer paper tailored towards an expert in the field.

### Response (temperature = 0.9):

The article provides a comprehensive overview of current cancer treatments and ongoing research in the field. It emphasizes the success of targeted therapy, which has been effective in treating specific types of cancers such as chronic myeloid leukemia and gastrointestinal stromal tumors. The use of Tamoxifen for ER-positive breast cancers is also discussed. The paper highlights emerging areas of research, including biological therapies like MABs, gene therapies, and cancer vaccines. Additionally, the article explores anti-angiogenic drugs that prevent cancers from developing the blood vessels they need to grow. The importance of immunotherapy is also highlighted, with recent advances such as the use of immunomodulatory MABs, adoptive transfer of activated T cells, and cancer vaccines. The paper concludes by stating that despite significant advancements, curing most cancers - particularly Stage IV metastatic disease - remains a distant goal.

### Response (temperature = 0.6):

The paper provides a comprehensive overview of various targeted therapies for cancer treatment, including small molecule tyrosine kinase inhibitors and monoclonal antibodies (MABs) such as Imatinib and Tamoxifen. The researchers are also exploring new and more efficient surgery techniques, radiotherapy and chemotherapy drugs, biological therapies such as MABs, cancer vaccines, and gene therapies, and anti-angiogenic drugs that can inhibit the growth of blood vessels that tumors require. Immunotherapy has recently shown promising results in improving survival and response durability in patients with different cancer types. However, the paper highlights that while early stage cancer treatment has been successful, Stage IV metastatic disease, particularly solid tumors, remains challenging to cure. The paper does not disclose any financial support or conflicts of interest.

### Response (temperature = 0.3):

The paper provides a critical analysis of the current state of cancer diagnosis and treatment. It highlights the success of conventional approaches such as surgery, radiotherapy, and chemotherapy in treating cancer. The paper notes that early detection is key to successful treatment and emphasizes the importance of screening tests such as mammograms, colonoscopies, and Pap smear examination. The authors also mention other conventional techniques used in the treatment of cancer including bone marrow transplantation, peripheral stem cell transplantation, hormone therapy, photodynamic therapy, cryosurgery, immunotherapy, and gene therapy. The paper acknowledges that there is still a long way to go before most cancers can be cured, particularly in the case of Stage IV metastatic disease. The authors conclude by emphasizing the need for continued research and development in the field to improve the diagnosis and treatment of cancer.

## Prompt 3: I am 9. Explain how doctors can manage my pain. Please use simple words so that I can understand.

### Response (temperature = 0.9):

Doctors can help you feel better when you have pain by understanding how much pain you are feeling. They might ask you to describe how your pain feels, or look at how you are moving, if you are crying or making sounds, or even your heart rate and blood pressure. Then, they can give you medicine or use other ways to make the pain go away. Sometimes, doctors will also want to know how your sleep, social relations, and school activities are going to make sure they are helping your whole body feel better.

### Response (temperature = 0.6):

Doctors can help you feel better when you are in pain by using different ways to measure how much pain you are feeling. They might look at how you react, like if you cry or make certain facial expressions, or check your body's responses, like your heart rate or blood pressure. They might also ask you to describe your pain. Depending on how old you are and how well you can talk about your pain, they might use different tools to help them understand how much pain you are feeling. Once they know how much pain you are in, they can give you medicine or other treatments to help you feel better. It's important for doctors to make sure they understand how much pain you are feeling so that they can give you the right treatment.

### Response (temperature = 0.3):

Doctors can help you feel better by finding out how much pain you are feeling. They may look at how your body is reacting or ask you to describe your pain. Based on this information, they can give you medicine or other treatments to make you feel better. It's important for doctors to also consider how pain affects your sleep, social life, and school activities to make sure they are treating your pain in the best way possible.

## Conclusion

Analysing the above responses, I believe that the model gives the best responses with a temperature of 0.9. With a temperature of 0.9, the model generates more relevent and insightful answers, is less prone to copying the helping prompts (unlike in prompt 1 with temperature = 0.3), and can better adjust it's tone and language to according to the task at hand.

For instance, in prompt 2, with temperature = 0.9, the model only provides information that a cancer expert will likely be interested in, but with temperature = 0.6, the model states that "the paper does not disclose any financial support or conflicts of interest", which an expert would not be interested in when asking for a summary of the paper.


# Citations

Roy, P. S., & Saikia, B. J. (2016). Cancer and cure: A critical analysis. _Indian journal of cancer_, 53(3), 441–442. [https://doi.org/10.4103/0019-509X.200658](https://doi.org/10.4103/0019-509X.200658)

Pongrac Barlovic, D., Harjutsalo, V., & Groop, P. H. (2022). Exercise and nutrition in type 1 diabetes: Insights from the FinnDiane cohort. _Frontiers in endocrinology_, 13, 1064185. [https://doi.org/10.3389/fendo.2022.1064185](https://doi.org/10.3389/fendo.2022.1064185)

Di Sarno, L., Gatto, A., Korn, D., Pansini, V., Curatola, A., Ferretti, S., Capossela, L., Graglia, B., & Chiaretti, A. (2023). Pain management in pediatric age. An update. _Acta bio-medica : Atenei Parmensis_, 94(4), e2023174. [https://doi.org/10.23750/abm.v94i4.14289](https://doi.org/10.23750/abm.v94i4.14289)
